In [1]:
import pandas as pd
import selenium
from selenium import webdriver
from bs4 import BeautifulSoup
import time
from tqdm import tnrange
from urllib.request import urlopen
import re
import requests
import urllib.request
from tqdm import tqdm

In [2]:
train = pd.read_csv("./data/train.csv")

In [3]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(chrome_options=options)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: use options instead of chrome_options
  """


In [ ]:
def backtranslation_aug(driver):
    train = pd.read_csv("./data/train.csv")

    # aug_cat extract
    aug_cat = []
    val = train["cat3"].values.tolist()
    val = list(set(val))

    for v in val:
        if len(train[train["cat3"] == v]) < 200:
            aug_cat.append(v)
    
    for idx in tqdm(range(len(train))):
        if train["cat3"][idx] in aug_cat:
            aug_view